# 丢弃法
除了前⼀节介绍的权重衰减以外，深度学习模型常常使⽤丢弃法（dropout） [1] 来应对过拟合问
题。丢弃法有⼀些不同的变体。本节中提到的丢弃法特指倒置丢弃法（inverted dropout）
## 方法
回忆⼀下，[ “多层感知机”](3.8mlp.ipynb) ⼀节的图描述了⼀个单隐藏层的多层感知机。其中输⼊个数为4，隐
藏单元个数为5，且隐藏单元$h_i(i = 1,...,5)$的计算表达式为

$$h_i=\phi(x_1w_{1i}+ x_2w_{2i}+x_3w_{3i}+x_4w_{4i}+b_i)$$

这⾥$\phi$是激活函数， $x_1,\ldots, x_4$是输⼊，隐藏单元i的权重参数为$w_{1i},\ldots,w_{4i}$，偏差参数为$b_i$。
当对该隐藏层使⽤丢弃法时，该层的隐藏单元将有⼀定概率被丢弃掉。设丢弃概率为p，那么有p的概
率$h_i$会被清零，有1 − p的概率$h_i$会除以1 − p做拉伸。丢弃概率是丢弃法的超参数。具体来说，设
随机变量$\xi_i$为0和1的概率分别为p和1 − p。使⽤丢弃法时我们计算新的隐藏单元$h'_i$
$$h'_i = \frac{\xi_i}{1-p}h $$

由于$E(\xi_i) = 1 − p$，因此$$E(h'_i) = \frac{E(\xi_i)}{1-p}h = h_i$$
即丢弃法不改变其输⼊的期望值。让我们对图中的隐藏层使⽤丢弃法，⼀种可能的结果如
图所⽰，其中h2和h5被清零。这时输出值的计算不再依赖h2和h5，在反向传播时，与这两个隐
藏单元相关的权重的梯度均为0。由于在训练中隐藏层神经元的丢弃是随机的，即$h_1,\ldots,h_5$都有
可能被清零，输出层的计算⽆法过度依赖$h_1,\ldots,h_5$中的任⼀个，从而在训练模型时起到正则化的
作⽤，并可以⽤来应对过拟合。在测试模型时，我们为了拿到更加确定性的结果，⼀般不使⽤丢
弃法。

![隐藏层使用了丢弃法的多层感知机](../img/dropout.svg)

## 从零开始实现
根据丢弃法的定义，我们可以很容易地实现它。下⾯的dropout函数将以drop_prob的概率丢
弃NDArray输⼊X中的元素。

In [1]:
import d2lzh as d2l
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn

def dropout(X, drop_prob):
    assert 0 <= drop_prob <= 1
    keep_prob = 1 - drop_prob
    # 这种情况下把全部元素都丢弃
    if keep_prob == 0:
        return X.zeros_like()
    mask = nd.random.uniform(0, 1, X.shape) < keep_prob
    return mask * X / keep_prob

我们运⾏⼏个例⼦来测试⼀下dropout函数。其中丢弃概率分别为0、 0.5和1。

In [2]:
X = nd.arange(16).reshape((2, 8))
dropout(X, 0)


[[ 0.  1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14. 15.]]
<NDArray 2x8 @cpu(0)>

In [3]:
dropout(X, 0.5)


[[ 0.  2.  4.  6.  0.  0.  0. 14.]
 [ 0. 18.  0.  0. 24. 26. 28.  0.]]
<NDArray 2x8 @cpu(0)>

In [4]:
dropout(X, 1)


[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 2x8 @cpu(0)>

### 定义模型参数
实验中，我们依然使⽤[“softmax回归的从零开始实现”](3.6softmax-regression-scratch.ipynb) ⼀节中介绍的Fashion-MNIST数据集。我
们将定义⼀个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数都是256。

In [5]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

W1 = nd.random.normal(scale=0.01, shape=(num_inputs, num_hiddens1))
b1 = nd.zeros(num_hiddens1)
W2 = nd.random.normal(scale=0.01, shape=(num_hiddens1, num_hiddens2))
b2 = nd.zeros(num_hiddens2)
W3 = nd.random.normal(scale=0.01, shape=(num_hiddens2, num_outputs))
b3 = nd.zeros(num_outputs)
params = [W1, b1, W2, b2, W3, b3]
for param in params:
    param.attach_grad()

### 定义模型
下⾯定义的模型将全连接层和激活函数ReLU串起来，并对每个激活函数的输出使⽤丢弃法。我
们可以分别设置各个层的丢弃概率。通常的建议是把靠近输⼊层的丢弃概率设得小⼀点。在这个
实验中，我们把第⼀个隐藏层的丢弃概率设为0.2，把第⼆个隐藏层的丢弃概率设为0.5。我们可
以通过[“⾃动求梯度”]() ⼀节中介绍的is_training函数来判断运⾏模式为训练还是测试，并只
需在训练模式下使⽤丢弃法。

In [6]:
drop_prob1, drop_prob2 = 0.2, 0.5
def net(X):
    X = X.reshape((-1, num_inputs))
    H1 = (nd.dot(X,W1) + b1).relu()
    if autograd.is_training():  # 只在训练模型时使⽤丢弃法
        H1 = dropout(H1, drop_prob1) # 在第⼀层全连接后添加丢弃层
    H2 = (nd.dot(H1,W2) + b2).relu()
    if autograd.is_training():
        H2 = dropout(H2, drop_prob2)
    return nd.dot(H2, W3) + b3

### 训练和测试模型
这部分与之前多层感知机的训练和测试类似。

In [7]:
batch_size, num_epochs, lr = 256, 5, 0.5
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
loss = gloss.SoftmaxCrossEntropyLoss()
d2l.train_ch3(net, train_iter, test_iter,loss, num_epochs,batch_size,params, lr)

epoch 1, loss 1.0880, train acc 0.575, test acc 0.786
epoch 2, loss 0.5717, train acc 0.785, test acc 0.837
epoch 3, loss 0.4868, train acc 0.822, test acc 0.846
epoch 4, loss 0.4447, train acc 0.837, test acc 0.847
epoch 5, loss 0.4144, train acc 0.849, test acc 0.871


## 简洁实现
在Gluon中，我们只需要在全连接层后添加Dropout层并指定丢弃概率。在训练模型时，
Dropout层将以指定的丢弃概率随机丢弃上⼀层的输出元素；在测试模型时， Dropout层并不
发挥作⽤。

In [8]:
net = nn.Sequential()
net.add(nn.Dense(256, activation='relu'),
        nn.Dropout(drop_prob1), # 在第⼀个全连接层后添加丢弃层
        nn.Dense(256, activation='relu'),
        nn.Dropout(drop_prob2),
        nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))# 在第二个全连接层后添加丢弃层

下⾯训练并测试模型。

In [9]:
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate': lr})
d2l.train_ch3(net,train_iter,test_iter,loss ,num_epochs,
              batch_size, None,None,trainer)

epoch 1, loss 1.0956, train acc 0.578, test acc 0.779
epoch 2, loss 0.5775, train acc 0.787, test acc 0.835
epoch 3, loss 0.4895, train acc 0.821, test acc 0.853
epoch 4, loss 0.4423, train acc 0.839, test acc 0.861
epoch 5, loss 0.4145, train acc 0.848, test acc 0.858


## 小结
- 我们可以通过使⽤丢弃法应对过拟合。
- 丢弃法只在训练模型时使⽤。